In [1]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time
import re
import hashlib

import sqlite3
import csv
import os

In [2]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    #working_df=working_df.drop_duplicates()
    print(f'working on {hashtag}')
    index=len(working_df)
    for tweet in tweets: 
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        #print(working_df.loc[index,'created_at'])
        #working_df=working_df.drop_duplicates()
        index+=1
        working_df.to_csv(f'source/{hashtag}.csv', index=False)
        #time.sleep(1)
    working_df=working_df.drop_duplicates()
    working_df.to_csv(f'source/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
          
    return working_df

In [5]:
def get_tweets_from_general_tags(working_df, hashtag):
    #load and merge the general_tags
    print(f'working on {hashtag}')
    general_tags = ['#SaveLockwoodandCo', 'Lockwood and Co']
    general_tags_df=pd.DataFrame()
    for tag in general_tags:
        try:
            general_tags_df=pd.concat([general_tags_df, pd.read_csv(f'source/{tag}.csv')], ignore_index=True)
        except:
            pass
    general_tags_df =general_tags_df.drop_duplicates()
    print(f'total generaltags: {len(general_tags_df)}')
    #general_tags_df['full_text'] = general_tags_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    #Get the rows in the working_df that have the hashtag inside the full_text column
    hashtag_df = general_tags_df[general_tags_df['full_text'].apply(lambda x: hashtag.lower() in x.lower())]
    #hashtag_df = hashtag_df.reset_index(drop=True)
    hashtag_df['hashtags'] = hashtag
    working_df=pd.concat([working_df, hashtag_df], ignore_index=True)
    working_df=working_df.drop_duplicates()
    working_df['full_text'] = working_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    working_df.to_csv(f'source/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
  
    return working_df

In [6]:
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [7]:
hashtagsweek1 = ['#FridayNightatPortlandRow', '#HauntedWatchParty', '#WatchPartyatPortlandRow', '#HauntedbyaType3','#TogetherForLockwoodandCo','#PrimeForLockwoodandCo','#BringBackLockwoodandCo']
hashtagsweek2 = ['#GhostHuntersWatchParty', '#DisneyForLockwoodandCo', '#BBCforLockwoodandCo', '#AppleTVforLockwoodandCo', '#PrimeForLockwoodandCo', '#JustRecklessEnough']
hashtagsweek3 = ['#LockwoodGhostAuditions', '#ParamountForLockwoodandCo', '#ScullandCo','#RapiersReady', '#CaringforCarlyle', '#DEPRACisOnTheWay', '#BunsForBunchurch']
hashtagsweek4 = ['#CompleteFictionAppreciation', '#DisneySaveLockwood', '#ArtistryofLockwoodandCo', '#ParamountSaveLockwood', '#GhostStrike', '#LockwoodParallelFandoms', '#JustRecklessEnough']
hashtagsweek5 = ['#StroudsAppreciation', '#VoteLockwoodforNFA', '#PrimeSaveLockwood', '#ScreamingStaircase', '#DEPRACrollcall', '#LivingforLockwood', '#RapiersReady']
hashtagsweek6 = ['#LockNationAppreciation', '#LockNationArtistsandGiftsDay', '#LockNationEditorsDay', '#LockNationFicWritersDay', '#LockNationComediansDay', '#GhostLockAwards']
hashtagsall = hashtagsweek1+hashtagsweek2+hashtagsweek3+hashtagsweek4 + hashtagsweek5
hashtagsall = list(set(hashtagsall))

In [8]:

phrase = 'Lockwood and Co'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=phrase, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)

hashtag = '#SaveLockwoodandCo'
try:
    working_df = pd.read_csv(f'source/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)


16243
working on Lockwood and Co


Rate limit reached. Sleeping for: 157
/tmp/ipykernel_899/1434055682.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)


created_at
2023-06-02 00:00:00+00:00      64
2023-06-03 00:00:00+00:00     313
2023-06-04 00:00:00+00:00     275
2023-06-05 00:00:00+00:00     443
2023-06-06 00:00:00+00:00     725
2023-06-07 00:00:00+00:00     922
2023-06-08 00:00:00+00:00    1159
2023-06-09 00:00:00+00:00    1710
2023-06-10 00:00:00+00:00    1937
2023-06-11 00:00:00+00:00    2756
2023-06-12 00:00:00+00:00     407
2023-06-13 00:00:00+00:00       0
2023-06-14 00:00:00+00:00       0
2023-06-15 00:00:00+00:00    4878
2023-06-16 00:00:00+00:00    4023
2023-06-17 00:00:00+00:00    2049
2023-06-18 00:00:00+00:00    2315
Freq: D, Name: hashtags, dtype: int64
22673
23976
working on Lockwood and Co
created_at
2023-06-02 00:00:00+00:00      64
2023-06-03 00:00:00+00:00     313
2023-06-04 00:00:00+00:00     275
2023-06-05 00:00:00+00:00     443
2023-06-06 00:00:00+00:00     725
2023-06-07 00:00:00+00:00     922
2023-06-08 00:00:00+00:00    1159
2023-06-09 00:00:00+00:00    1710
2023-06-10 00:00:00+00:00    1937
2023-06-11 00:00:

In [8]:
hashtags = ['#PrimeSaveLockwood']
for hashtag in hashtagsweek6:
    try:
        working_df=pd.read_csv(f'source/{hashtag}.csv')
        #working_df = pd.read_csv('source/#SkullandCo.csv')
        print(len(working_df))
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
    tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
    working_df = get_tweets_dataframe(working_df, tweets, hashtag)
    working_df=working_df.drop_duplicates()
    print(len(working_df))
    time.sleep(60)

209
working on #LockNationAppreciation
created_at
2023-06-18 00:00:00+00:00       2
2023-06-19 00:00:00+00:00    3723
2023-06-20 00:00:00+00:00     628
2023-06-21 00:00:00+00:00      23
2023-06-22 00:00:00+00:00       2
Freq: D, Name: hashtags, dtype: int64
4288
0
working on #LockNationArtistsandGiftsDay
created_at
2023-06-20 00:00:00+00:00    1
Freq: D, Name: hashtags, dtype: int64
1
0
working on #LockNationEditorsDay


Rate limit reached. Sleeping for: 439


created_at
2023-06-20 00:00:00+00:00      45
2023-06-21 00:00:00+00:00    2905
2023-06-22 00:00:00+00:00     490
Freq: D, Name: hashtags, dtype: int64
3440
0
working on #LockNationFicWritersDay
Series([], Freq: D, Name: hashtags, dtype: int64)
0
0
working on #LockNationComediansDay
Series([], Freq: D, Name: hashtags, dtype: int64)
0
0
working on #GhostLockAwards
Series([], Freq: D, Name: hashtags, dtype: int64)
0


In [9]:
hashtags = ['#SkullandCo']
for hashtag in hashtagsall:
    try:
        
        working_df=pd.read_csv(f'source/{hashtag}.csv')
        print('try')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
        print('except')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
        

try
working on #ArtistryofLockwoodandCo
total generaltags: 77004
created_at
2023-06-06 00:00:00+00:00      2
2023-06-07 00:00:00+00:00    933
2023-06-08 00:00:00+00:00    995
2023-06-09 00:00:00+00:00     62
2023-06-10 00:00:00+00:00     10
2023-06-11 00:00:00+00:00     29
2023-06-12 00:00:00+00:00      7
2023-06-13 00:00:00+00:00      3
2023-06-14 00:00:00+00:00      1
2023-06-15 00:00:00+00:00      2
2023-06-16 00:00:00+00:00      1
Freq: D, Name: hashtags, dtype: int64
try
working on #LivingforLockwood


/tmp/ipykernel_712/2862647720.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashtag_df['hashtags'] = hashtag


total generaltags: 77004
created_at
2023-06-16 00:00:00+00:00      38
2023-06-17 00:00:00+00:00    5122
2023-06-18 00:00:00+00:00    1059
2023-06-19 00:00:00+00:00      23
Freq: D, Name: hashtags, dtype: int64
try
working on #WatchPartyatPortlandRow
total generaltags: 77004
created_at
2023-05-20 00:00:00+00:00       1
2023-05-21 00:00:00+00:00    3902
2023-05-22 00:00:00+00:00    1243
2023-05-23 00:00:00+00:00      92
2023-05-24 00:00:00+00:00       8
2023-05-25 00:00:00+00:00       1
2023-05-26 00:00:00+00:00       0
2023-05-27 00:00:00+00:00       0
2023-05-28 00:00:00+00:00       2
2023-05-29 00:00:00+00:00       0
2023-05-30 00:00:00+00:00       3
2023-05-31 00:00:00+00:00       0
2023-06-01 00:00:00+00:00       2
2023-06-02 00:00:00+00:00       0
2023-06-03 00:00:00+00:00       0
2023-06-04 00:00:00+00:00       0
2023-06-05 00:00:00+00:00       0
2023-06-06 00:00:00+00:00       0
2023-06-07 00:00:00+00:00       0
2023-06-08 00:00:00+00:00       1
Freq: D, Name: hashtags, dtype: in

In [79]:
working_df=pd.DataFrame()
for hashtag in hashtagsall:
    try:
        working_df=pd.concat([working_df, pd.read_csv(f'source/{hashtag}.csv')], ignore_index=True)
    except:
        pass
len(working_df)

210522

In [80]:
working_df['full_text'] = working_df['full_text'].str.replace(r'"', '').replace(r'\n', ' ').replace(':', ' ').replace(',', '').replace('!', '')

In [81]:
# Create a new column 'retweet' with default value False
working_df['retweet'] = False

# Check if 'full_text' starts with 'RT ' and set 'retweet' column accordingly
working_df.loc[working_df['full_text'].str.startswith('RT '), 'retweet'] = True

#Remove the 'RT ' in the full_text column
working_df['full_text'] = working_df['full_text'].str.replace('RT ', '')

working_df['retweet'].value_counts()

False    116660
True      93862
Name: retweet, dtype: int64

In [82]:
def get_week_label(week_number):
    if week_number == 20:
        return 'week1'
    elif week_number == 21:
        return 'week2'
    elif week_number == 22:
        return 'week3'
    elif week_number == 23:
        return 'week4'
    elif week_number == 24:
        return 'week5'
    elif week_number == 25:
        return 'week6'
    else:
        return 'week1'


In [83]:
working_df['created_date'] = pd.to_datetime(working_df['created_at'], utc=True)
working_df['week'] = working_df['created_date'].dt.isocalendar().week
working_df['week'] = working_df['week'].apply(get_week_label)
working_df['week'].value_counts()

week4    47107
week1    45386
week3    45240
week2    36116
week5    36102
week6      571
Name: week, dtype: int64

In [84]:
#Create an anonymized tweet_id
#working_df['unique_id'] = str(working_df['tweet_id']) + working_df['created_at'] + working_df['user'] + str(working_df['retweet'])
working_df['unique_id'] = working_df.apply(lambda row: str(row['tweet_id']) + row['created_at'] + row['user'] + str(row['retweet']), axis=1)
working_df['tweet_id'] = working_df['unique_id'].apply(anonymize)

In [85]:
hashtags_df = working_df[['tweet_id','hashtags']]
hashtags_df.to_csv('output/hashtags.csv', index=False)
working_df.value_counts('hashtags')

hashtags
#JustRecklessEnough             18875
#PrimeForLockwoodandCo          15244
#RapiersReady                   14085
#ParamountForLockwoodandCo       9950
#TogetherForLockwoodandCo        9415
#BunsForBunchurch                9279
#ParamountSaveLockwood           9196
#CompleteFictionAppreciation     8818
#BringBackLockwoodandCo          7853
#LockwoodParallelFandoms         7758
#ScreamingStaircase              7174
#LockwoodGhostAuditions          6464
#GhostStrike                     6349
#CaringforCarlyle                6289
#LivingforLockwood               6242
#DisneyForLockwoodandCo          5858
#HauntedbyaType3                 5811
#DEPRACisOnTheWay                5419
#BBCforLockwoodandCo             5377
#WatchPartyatPortlandRow         5255
#FridayNightatPortlandRow        5213
#DEPRACrollcall                  5068
#DisneySaveLockwood              5021
#HauntedWatchParty               4838
#AppleTVforLockwoodandCo         4746
#VoteLockwoodforNFA              4353
#Gh

In [86]:
# Get the tagged users from the full_text
working_df=working_df.drop_duplicates('tweet_id')
filter_out = ['@AppleSupport ','@AppleTV ','@AppleTVI ','@AppleTVPlus ','@AppleTVUK ','@BBC ', '@BBCOne ', '@BBCPlayer ','@BBCSounds ', '@BBCR1 ',
            '@BBCSounds ','@BBCiPlayer ','@BBCone ','@D ','@Di ','@Dis ','@Disn ', '@Disne ', '@Dinsey ','@DisneyChannel ','@DisneyHyperion ','@DisneyIT ', '@DisneyPlus ', '@DisneyChannel ',
            '@DisneyHyperion ','@DisneyPlusUK', '@DisneyPlus', '@HBO_UK ','@HBO ', '@hulu ','@NETFLIX ', '@netflix ', '@NETFLIXUK ', '@Netflix ', '@NetflixUK ','@ParamountPlus ', '@ParamaountUK ', '@ParamountPlusUK '
            '@Prime ', '@PrimeUK ', '@PrimeVideo ', '@PrimeVideo','@primevideouk ']
for filter in filter_out:
    working_df['full_text'] = working_df['full_text'].str.replace(filter, '')
working_df['tagged_users'] = working_df['full_text'].apply(lambda x: re.findall(r'@(\w+)', x))
# Remove the tagged users from the full_text
working_df['full_text'] = working_df['full_text'].str.replace(r'@(\w+)', '')
list_of_tagged_users = []
for x in range(len(working_df)):
    try:
        list_of_tagged_users += working_df['tagged_users'][x]
    except:
        pass



/tmp/ipykernel_712/3040538264.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  working_df['full_text'] = working_df['full_text'].str.replace(r'@(\w+)', '')


In [87]:
#Create a table for network analysis
na_df = working_df[['tweet_id','user', 'retweet', 'tagged_users']]

In [88]:
#Create a function that would create a dataframe that shows the interaction
def get_interaction(df):
    func_df = pd.DataFrame(columns = ['tweet_id', 'from_', 'to_'])
    filter_out = ['2','A','App','AppleSupport','AppleTV','AppleTVI','AppleTVPlus','AppleTVUK','BB','BBC','BBCO','BBC','BBCOn', 'BBCOne', 'BBCPlayer','BBCSounds', 'BBCR1',
                  'BBCSounds','BBCiPlayer','BBCone','D','Di','Dis','Disn', 'Disne', 'Dinsey',' DisneyChannel','DisneyHyperion','DisneyIT', 'DisneyP', 'DisneyChannel',
                  'DisneyHyperion','DisneyPlusUK', 'DisneyPlus', 'HBO', 'HBO_UK','NETFLIX', 'netflix', 'NETFLIXUK', 'Netflix', 'NetflixUK','ParamountPlus', 'ParamaountUK', 'ParamountPlusUK'
                  'Prime', 'PrimeUK', 'PrimeVideo', 'primevideouk']
    for x in range(len(df)):
        if df['retweet'][0]=='True':
            if df['tagged_users'].iloc[x][0] not in filter_out:
                new_row = {'tweet_id':df['tweet_id'].iloc[x], 'from_':df['tagged_users'].iloc[x][0], 'to_': df['user'].iloc[x]}
                func_df = func_df.append(new_row, ignore_index = True)
            else:
                print(df['tagged_users'].iloc[x][0])
        else:
            for user in df['tagged_users'].iloc[x]:
                if user not in filter_out:
                    new_row = {'tweet_id':df['tweet_id'].iloc[x], 'to_':user, 'from_': df['user'].iloc[x]}
                    func_df = func_df.append(new_row, ignore_index = True)
                else:
                    print(user)
    return func_df.drop_duplicates()


In [89]:
filter_out = ['2','A','App','AppleSupport','AppleTV','AppleTVI','AppleTVPlus','AppleTVUK','BB','BBC','BBCO','BBC','BBCOn', 'BBCOne', 'BBCPlayer','BBCSounds', 'BBCR1',
                  'BBCSounds','BBCiPlayer','BBCone','D','Di','Dis','Disn', 'Disne', 'Dinsey',' DisneyChannel','DisneyHyperion','DisneyIT', 'DisneyP', 'DisneyChannel',
                  'DisneyHyperion','DisneyPlusUK', 'DisneyPlus', 'HBO', 'HBO_UK','NETFLIX', 'netflix', 'NETFLIXUK', 'Netflix', 'NetflixUK','ParamountPlus', 'ParamaountUK', 'ParamountPlusUK'
                  'Prime', 'PrimeUK', 'PrimeVideo', 'primevideouk']
na_interac_df = na_interac_df[~na_interac_df['from_'].isin(filter_out)]
na_interac_df = na_interac_df[~na_interac_df['to_'].isin(filter_out)]                            
print(na_interac_df[na_interac_df['from_'].isin(filter_out)])
print(na_interac_df[na_interac_df['to_'].isin(filter_out)])

Empty DataFrame
Columns: [tweet_id, from_, to_]
Index: []
Empty DataFrame
Columns: [tweet_id, from_, to_]
Index: []


In [90]:
#get a unique user dataframe
list_of_user = na_interac_df['from_'].append(na_interac_df['to_'], ignore_index=True)
list_of_user = list(set(list_of_user))
user_df = pd.DataFrame(list_of_user, columns=['username'])
user_df['user'] = user_df['username'].apply(anonymize)
user_df=user_df.sort_values('username').reset_index()
user_df.to_csv('output/username.csv', index=False)

In [91]:
#anonymise the users from the different dataframes
na_interac_df['from_'] = na_interac_df['from_'].map(user_df.set_index('username')['user'])
na_interac_df['to_'] = na_interac_df['to_'].map(user_df.set_index('username')['user'])
working_df['user'] = working_df['user'].map(user_df.set_index('username')['user'])
na_interac_df.to_csv('output/userinteraction.csv', index=False)

In [92]:
working_df = working_df[['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'retweet', 'week']]
working_df.to_csv('output/tweets.csv', index=False)

In [93]:
set(hashtags_df['tweet_id'])==set(working_df['tweet_id'])

True

In [94]:
# remove existing database if available
if os.path.exists('output/lnctweets.db'):
    os.remove('output/lnctweets.db')
# Creating a database file for the all the output csv file
# Establish a connection to the SQLite Database
conn = sqlite3.connect('output/lnctweets.db')
# Create a cursor object to execute SQL statements:
cursor = conn.cursor()

In [95]:
def create_table(table_name, columns):
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({', '.join(columns)})"
    cursor.execute(create_table_query)

In [96]:
# Define a function to insert data into the table from a CSV file:
def insert_data(table_name, csv_file):
    with open(csv_file, 'r') as file:
        csv_data = csv.reader(file)
        next(csv_data)  # Skip the header row if necessary
        num_columns = len(next(csv_data))
        insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * num_columns)})"
        cursor.executemany(insert_query, csv_data)

In [97]:
# Specify the CSV file paths and table names for each file:
csv_files = ['output/hashtags.csv',  'output/tweets.csv', 'output/userinteraction.csv']
table_names = ['hashtags', 'tweets', 'userinteraction']

In [98]:
# Create tables and insert data for each CSV file:
for i, csv_file in enumerate(csv_files):
    table_name = table_names[i]
    with open(csv_file, 'r') as file:
        csv_data = csv.reader(file)
        columns = next(csv_data)
        data_type = ' TEXT'
        columns = [x + data_type for x in columns]
        #print(columns)
        create_table(table_name, columns)
        insert_data(table_name, csv_file)

In [99]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [100]:
# open the tweets table in the .db file
query = """
    SELECT *
    FROM userinteraction
    """
conn = sqlite3.connect('output/lnctweets.db')
print(pd.read_sql_query(query,conn))
conn.close()

        tweet_id     from_       to_
0       6cbe5687  41c6f32a  fe802c03
1       6cbe5687  41c6f32a  bdc181f0
2       6cbe5687  41c6f32a  d4ced168
3       6cbe5687  41c6f32a  20d15a56
4       f8842111  0cfbd6ef  707e67d9
...          ...       ...       ...
141306  270964a5  6fc88075  20a79d68
141307  4d269ca7  47994202  519ba526
141308  dd27ac98  d8d26b48  ebcc1271
141309  a6376cfa  432fc05e  d14c58ed
141310  a6376cfa  432fc05e  20d15a56

[141311 rows x 3 columns]
